In [1]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    # api_key="sk-0rLvuRkMiD4Mw25QYygh6rUlZVjpQWNGNF4yez7z3PZ7yCOm",
    api_key="sk-tMbkq3K1iO5vf0FRMlrmzslGXJZwE0us3mve4QXuvpnZcumG",
    base_url="https://api.chatanywhere.tech/v1"
    # base_url="https://api.chatanywhere.cn/v1"
)

In [2]:
def gpt_35_api(messages: list):

    completion = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=messages, temperature=0.5, max_tokens=20)
    return completion.choices[0].message.content

In [3]:
import json
from tqdm import tqdm

In [5]:
from langchain import hub

prompt = hub.pull("tyfann/llm4commit-cot")

In [6]:
with open('../data/test_data_with_rag_100.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

In [8]:
example_q = """"diff --git a/java/client/src/org/openqa/selenium/htmlunit/HtmlUnitAlert.java  b/java/client/src/org/openqa/selenium/htmlunit/HtmlUnitAlert.java \npublic void sendKeys(String keysToSend){ \npublic void authenticateUsing(Credentials credentials){}+@Override+public void setCredentials(Credentials credentials){+}++@Override \npublic void handleAlert(Page page,String message){ \nQueue<String>queue=queues.get(page); \n" What is the commit message of this code diff?"""
example_a = """First, the scope of code changes is to add a method, and second, according to the filename "HtmlUnitAlert", and the function of the file could be an alert, the modification "@Overide" is an annotation which can solve the compilation issue, so the modification can help solve the compilation problem to the alert file. So the commit message for the code diff is "fix compilation issue because of new method in Alert"."""
gpt_msg = []
for data in tqdm(org_data, total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"Example_Question": example_q, "Example_Answer": example_a, "Question": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    gpt_msg.append(gpt_35_api(example_prompt))

Processing documents:   0%|          | 0/100 [00:00<?, ?it/s]

python(74422) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Processing documents: 100%|██████████| 100/100 [02:46<00:00,  1.67s/it]


In [14]:
gpt_msg[1]

'The commit message for this code diff could be "Add validation for configuring both zookeeper and etcd'

In [16]:
import re

def preprocess(text):
    if re.search(r'commit\s*message', text, re.IGNORECASE):
        pattern = r'\"([^"]+)'
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        else:
            return text
    else:
        return text

In [17]:
with open('../data/msg_nngen_nmt_codebert_chatgpt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

for item, msg in zip(org_data[:100], gpt_msg):
    item['chatgpt_cot'] = preprocess(msg)

output_file = '../data/chatgpt_cot.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data[:100], f, ensure_ascii=False, indent=4)

In [28]:
max_len = 0
for msg in gpt_msg:
    max_len = max(max_len, len(msg.split()))

提取的内容: Refactor drawExtras method in PieChartRenderer to remove unnecessary parameter in drawBitmap
